In [1]:
# !kaggle competitions download -c digit-recognizer




  0%|          | 0.00/15.3M [00:00<?, ?B/s]
 13%|█▎        | 2.00M/15.3M [00:00<00:01, 13.4MB/s]
 26%|██▌       | 4.00M/15.3M [00:00<00:00, 15.5MB/s]
 39%|███▉      | 6.00M/15.3M [00:00<00:01, 5.70MB/s]
 52%|█████▏    | 8.00M/15.3M [00:01<00:00, 7.84MB/s]
 65%|██████▌   | 10.0M/15.3M [00:01<00:00, 10.0MB/s]
 78%|███████▊  | 12.0M/15.3M [00:01<00:00, 11.9MB/s]
 91%|█████████▏| 14.0M/15.3M [00:01<00:00, 13.5MB/s]
100%|██████████| 15.3M/15.3M [00:01<00:00, 11.1MB/s]


In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
import numpy as np
# import mlflow

# Loading data

In [2]:
# Loading train data
train_df = pd.read_csv('data/train.csv')
train_df.head()

In [4]:
# Extract label from train data
y = train_df['label']
X = train_df.drop(columns=['label'])
X.shape

In [6]:
# Split the train data into TRAIN and VALID datasets
X_train, X_valid, y_train_classes, y_valid_classes = train_test_split(X, y, test_size=0.25, random_state=0 )

In [7]:
# OneHotEncoding of the digits labels to match the Neural Network output
y_full = pd.get_dummies(y)
y_train = pd.get_dummies(y_train_classes)
y_valid = pd.get_dummies(y_valid_classes)

In [8]:
X_test = pd.read_csv('data/test.csv')
X_test.head()

pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  
3         0         0         0         0  
4         0         0         0         0  

[5 rows x 784 columns]

# Regular Neural Network

In [8]:
model = tf.keras.models.Sequential([
    Dense(128, activation='relu', input_shape=(784,)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(10, activation="softmax")
])

In [9]:

loss_function = 'categorical_crossentropy'
optimizer =  'adam'

model.compile(loss=loss_function, optimizer=optimizer,
              metrics=["accuracy"])

In [18]:
epochs = 100
batch_size = 32

history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_valid, y_valid))

Epoch 1/100
985/985 [==============================] - 2s 3ms/step - loss: 0.2628 - accuracy: 0.9295 - val_loss: 0.2001 - val_accuracy: 0.9492
Epoch 2/100
985/985 [==============================] - 3s 3ms/step - loss: 0.2626 - accuracy: 0.9313 - val_loss: 0.2061 - val_accuracy: 0.9484
Epoch 3/100
985/985 [==============================] - 4s 4ms/step - loss: 0.2378 - accuracy: 0.9368 - val_loss: 0.1856 - val_accuracy: 0.9525
Epoch 4/100
985/985 [==============================] - 4s 4ms/step - loss: 0.2340 - accuracy: 0.9377 - val_loss: 0.1895 - val_accuracy: 0.9522
Epoch 5/100
985/985 [==============================] - 4s 4ms/step - loss: 0.2353 - accuracy: 0.9375 - val_loss: 0.1877 - val_accuracy: 0.9530
Epoch 6/100
985/985 [==============================] - 4s 4ms/step - loss: 0.2220 - accuracy: 0.9417 - val_loss: 0.1999 - val_accuracy: 0.9459
Epoch 7/100
985/985 [==============================] - 3s 3ms/step - loss: 0.2109 - accuracy: 0.9440 - val_loss: 0.1752 - val_accuracy: 0.9560

In [20]:
# Compute confusion matrix for VALID dataset
valid_preds = np.argmax(model.predict(X_valid),axis=-1)
print(tf.math.confusion_matrix(y_valid_classes, valid_preds))

tf.Tensor(
[[ 983    0    2    2    0    0    8    1   19    0]
 [   0 1173    7    2    2    0    2    0    4    0]
 [   3    1 1042    7    0    0    2    8   13    1]
 [   0    1    9 1018    0   11    0    9   17    5]
 [   3    6    1    0  998    1    5    1    6   13]
 [   1    0    1    9    1  897    6    1   12    2]
 [   1    1    0    0    0    4 1033    0    5    0]
 [   3    6   12    0    3    2    0 1095    1    7]
 [   4    8    6    7    2    7   14    0  936   11]
 [   4    3    3    7   10    9    0   19   15  946]], shape=(10, 10), dtype=int32)


In [30]:
# Prepare submission CSV filed
submission = pd.read_csv('data/sample_submission.csv', index_col='ImageId')
test_preds = np.argmax(model.predict(X_test),axis=-1) # predictions on TEST dataset
submission['Label'] = test_preds
submission.head()

Label
ImageId       
1            0
2            0
3            0
4            0
5            0

In [32]:
# Export submission to CSV
submission.to_csv('data/submission_NN.csv')

# Convolution NN
To use Convolutional Neural Network, the dataset needs to be reshaped to the original 28x28 pixels images.

In [9]:
# Reshaping all datasets to suit CNN
arr_train = np.array(X_train)
arr_valid = np.array(X_valid)
arr_test = np.array(X_test)

X_train_arr = arr_train.reshape((arr_train.shape[0], 28, 28, 1))
X_valid_arr = arr_valid.reshape((arr_valid.shape[0], 28, 28,1 ))
X_test_arr = arr_test.reshape((arr_test.shape[0], 28, 28,1))


# Normalised data
X_train_norm = X_train_arr/255
X_valid_norm = X_valid_arr/255
X_test_norm = X_test_arr/255
# X_train_norm[0]

In [15]:

#### 100 epochs :0.97164 (on test), 0.9729 on valid
# CNN_model = tf.keras.models.Sequential([
#     Conv2D(2,3, activation='relu', input_shape=(28,28,1)),
#     MaxPooling2D((2,2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.3),
#     Dense(256, activation='relu'),
#     Dropout(0.3),
#     Dense(64, activation='relu'),
#     Dense(10, activation="softmax")
# ])

# 67 epochs : 0.97489 on test, 0.9772 on valid
# CNN_model = tf.keras.models.Sequential([
#     Conv2D(2,3, activation='relu', input_shape=(28,28,1)),
#     MaxPooling2D((2,2)),
#     Conv2D(2,3, activation='relu' ),
#     Flatten(),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dense(10, activation="softmax")
# ])


# 100 epochs Score : 0.9796 on valid
CNN_model = tf.keras.models.Sequential([
    Conv2D(2,2, activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Conv2D(2,2, activation='relu' ),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(10, activation="softmax")
])

In [16]:
CNN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 27, 27, 2)         10        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 2)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 2)         18        
_________________________________________________________________
flatten (Flatten)            (None, 288)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               147968    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2

In [17]:

loss_function = 'categorical_crossentropy'
optimizer =  'adam'

CNN_model.compile(loss=loss_function, optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
# Training model with original values (0-255)

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
epochs = 100
batch_size = 128
#best_valid:  0.98305
# simple early stopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Not used, allow to stop training if the validation loss does not decrease in a certain number of epochs ("patience" parameter)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True) # Save the model with best validation accuracy
# fit model
history = CNN_model.fit(X_train_arr,y_train, epochs=epochs, validation_data=(X_valid_arr, y_valid), callbacks=[mc])

In [19]:
# Training model with normalised data
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
epochs = 100
batch_size = 128
#best_valid:  0.98267 (100 epochs)
# simple early stopping
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model_norm.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
# fit model
history = CNN_model.fit(X_train_norm,y_train, epochs=epochs, validation_data=(X_valid_norm, y_valid), callbacks=[mc])

ss: 0.1268 - val_accuracy: 0.9820

Epoch 00002: val_accuracy improved from 0.97990 to 0.98200, saving model to best_model_norm.h5
Epoch 3/100
985/985 [==============================] - 18s 18ms/step - loss: 0.0154 - accuracy: 0.9957 - val_loss: 0.1232 - val_accuracy: 0.9815

Epoch 00003: val_accuracy did not improve from 0.98200
Epoch 4/100
985/985 [==============================] - 17s 17ms/step - loss: 0.0190 - accuracy: 0.9948 - val_loss: 0.1343 - val_accuracy: 0.9817

Epoch 00004: val_accuracy did not improve from 0.98200
Epoch 5/100
985/985 [==============================] - 17s 17ms/step - loss: 0.0175 - accuracy: 0.9949 - val_loss: 0.1452 - val_accuracy: 0.9809

Epoch 00005: val_accuracy did not improve from 0.98200
Epoch 6/100
985/985 [==============================] - 17s 17ms/step - loss: 0.0163 - accuracy: 0.9953 - val_loss: 0.1144 - val_accuracy: 0.9806

Epoch 00006: val_accuracy did not improve from 0.98200
Epoch 7/100
985/985 [==============================] - 17s 17ms/st

In [18]:
hist_df = pd.DataFrame(history.history)
hist_df.sort_values("val_accuracy", ascending=False)

NameError: name 'history' is not defined

In [20]:
# load the best model saved during fitting
from keras.models import load_model

model_names = ['best_model.h5', 'best_model_norm.h5']
current_model = model_names[0]
saved_model = load_model(current_model)


## Submission for original data

In [21]:
submission = pd.read_csv('data/sample_submission.csv', index_col='ImageId')
test_preds = np.argmax(saved_model.predict(X_test_arr),axis=-1)
submission['Label'] = test_preds
submission.to_csv('data/submission_CNN.csv')

Label
ImageId       
1            0
2            0
3            0
4            0
5            0

In [ ]:
# Compute confusion matrix for VALID dataset
valid_preds = np.argmax(saved_model.predict(X_valid_arr),axis=-1)
print(tf.math.confusion_matrix(y_valid_classes, valid_preds))

## Submission for normalised data

In [ ]:
submission = pd.read_csv('data/sample_submission.csv', index_col='ImageId')
test_preds = np.argmax(saved_model.predict(X_test_norm),axis=-1)
submission['Label'] = test_preds
submission.to_csv('data/submission_CNN_norm.csv')

In [ ]:
# Compute confusion matrix for VALID dataset
valid_preds = np.argmax(saved_model.predict(X_valid_norm),axis=-1)
print(tf.math.confusion_matrix(y_valid_classes, valid_preds))